In [1]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [2]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [3]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [4]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [5]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [6]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [7]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [8]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [9]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [10]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [11]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V5

In [12]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V6

In [13]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V7

In [14]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V8

In [15]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V9

In [16]:
#!CUDA_VISIBLE_DEVICES=1 python eval.py --config_name BASELINE_HF_V8FTEVAL

In [17]:
# def set_gpu_environ():
#     """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
#     Meant to be used in notebooks only.
#     """
#     import os
#     import subprocess
#     query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
#     utilization = [int(x.replace(" MiB", "")) for x in query]
#     free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
#     set_visible = ",".join([str(i) for i in free])
#     os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
# set_gpu_environ()

In [18]:
import os
import random
import numpy as np
import torch
import config

def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=cfg.WARM_UP_PCT * cfg.EPOCHS,
        num_training_steps=cfg.EPOCHS,
    )



    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )
    
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-02-26 18:20:52 - get_logger - Writing log to logs/graphnet_20230226-182052.log


In [ ]:
main('BASELINE_graph_V8_FT_3')

Training with config: {'__module__': 'config', 'MODEL_WTS': '/opt/slh/icecube/RESULTS/EXP_33_FT/EXP_33_FT_5.pth', 'EXP_NAME': 'EXP_33_FT_3_KAPPA', 'FILTER': True, 'LOSS_FUNC': <class 'icecube.modelsgraph.gVonMisesFisher3DLoss'>, 'NUM_WORKERS': 22, 'TRN_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'VAL_DATASET': <class 'icecube.graphdataset.GraphDasetV0'>, 'TRN_BATCH_RANGE': [[1, 50], [50, 100], [100, 150], [150, 200], [200, 250], [250, 300], [300, 350], [350, 400], [400, 450], [450, 500], [500, 550], [500, 600]], 'EPOCHS': 24, 'DEVICE': 'cuda:0', 'BATCH_SIZE': 756, 'LR': 0.0001, 'WD': 0.05, '__doc__': None}
Loading model weights from /opt/slh/icecube/RESULTS/EXP_33_FT/EXP_33_FT_5.pth


wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [1, 50]
Better model found at epoch 0 with value: 1.310355544090271.
   epoch  train_loss  valid_loss     metric
0      0    2.398044    2.397724  1.3103555
trn_range: [50, 100]
Better model found at epoch 1 with value: 1.3111745119094849.
   epoch  train_loss  valid_loss     metric
0      1    2.397161    2.397189  1.3111745
trn_range: [100, 150]
Better model found at epoch 2 with value: 1.3104790449142456.
   epoch  train_loss  valid_loss    metric
0      2    2.397225    2.397198  1.310479
trn_range: [150, 200]
Better model found at epoch 3 with value: 1.3111501932144165.
   epoch  train_loss  valid_loss     metric
0      3     2.39662    2.396823  1.3111502
trn_range: [200, 250]
Better model found at epoch 4 with value: 1.3105133771896362.
   epoch  train_loss  valid_loss     metric
0      4     2.39684    2.396413  1.3105134
trn_range: [250, 300]
Better model found at epoch 5 with value: 1.310032844543457.
   epoch  train_loss  valid_loss     metric
0      5    2.396449

In [ ]:
torch.cuda.current_device()

In [ ]:
k

In [ ]:
k